In [25]:
from langchain_core.tools import tool
import datetime

@tool
def add_two_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@tool
def get_current_time() -> str:
    """Get the current time"""
    return str(datetime.datetime.now())


In [26]:
add_two_numbers.name, add_two_numbers.description, add_two_numbers.args

('add_two_numbers',
 'Add two numbers together',
 {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}})

In [27]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-5-nano")

tools = [
    add_two_numbers, get_current_time
]

In [29]:
llm_with_tools = llm.bind_tools(tools)

In [48]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

chain  = llm_with_tools | JsonOutputToolsParser(tools=tools) 
response = chain.invoke("현재 시간을 알려주세요")
display(response)

[{'args': {}, 'type': 'get_current_time'}]

In [52]:
from IPython.display import display

response = chain.invoke("1234 + 5677")
display(response)


[{'args': {'a': 1234, 'b': 5677}, 'type': 'add_two_numbers'}]

In [53]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]
        tool_args = tool_call_result["args"]

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(f"[실행도구] {tool_name}\n[실행결과] {result}")
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")



In [54]:
execute_tool_calls(response)

[실행도구] add_two_numbers
[실행결과] 6911


In [56]:
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls
response = chain.invoke("1234 + 5677")
display(response)

[실행도구] add_two_numbers
[실행결과] 6911


None

In [57]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Agent프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [58]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [60]:
# Agent 실행
result = agent_executor.invoke({"input": "오늘의 날짜와 어제의 날짜를 구해보고, 일자의 더한 값을 알려주세요"})

# 결과 확인
print(result["output"])




> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`


2025-08-13 16:08:07.076321
Invoking: `get_current_time` with `{}`


2025-08-13 16:08:07.076566
Invoking: `add_two_numbers` with `{'a': 13, 'b': 12}`


25오늘의 날짜는 2025년 8월 13일이고, 어제의 날짜는 2025년 8월 12일입니다. 두 날짜의 일자를 더한 값은 25입니다.

> Finished chain.
오늘의 날짜는 2025년 8월 13일이고, 어제의 날짜는 2025년 8월 12일입니다. 두 날짜의 일자를 더한 값은 25입니다.
